In [1]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from torch import nn
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import utils as vutils
from torchvision import transforms, datasets

In [2]:
num_workers = os.cpu_count()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
save_dir = "./runs/WGAN"
data_dir = "/home/pervinco/Datasets/CelebA"

epochs = 100
batch_size = 128
img_size = 64
lr = 0.0001
beta1 = 0.5 
beta2 = 0.999

critic_iter = 5
clip_value = 0.01
c_lambda = 10

nc = 3
nz = 100
ngf = 64
ndf = 64

In [4]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [5]:
dataset = datasets.ImageFolder(root=data_dir,
                                transform=transforms.Compose([
                                    transforms.Resize(img_size),
                                    transforms.CenterCrop(img_size),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                ]))
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [6]:
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    if isinstance(m, nn.BatchNorm2d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
        torch.nn.init.constant_(m.bias, 0)


class Generator(nn.Module):
    def __init__(self, nz, ngf, nc):
        super().__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)


class Discriminator(nn.Module):
    def __init__(self, nc, ndf):
        super().__init__()
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            # nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [7]:
def save_fake_images(epoch, G, fixed_noise, num_images=64):
    with torch.no_grad():  # 그래디언트 계산을 하지 않음
        fake_images = G(fixed_noise).detach().cpu()
    fig = plt.figure(figsize=(8, 8))
    plt.axis("off")
    plt.title(f"Fake Images at Epoch {epoch}")
    plt.imshow(np.transpose(vutils.make_grid(fake_images[:num_images], padding=2, normalize=True), (1, 2, 0)))
    plt.savefig(f"{save_dir}/Epoch_{epoch}_Fake.png")  # 이미지 파일로 저장
    plt.close(fig)

In [8]:
fixed_noise = torch.randn(batch_size, nz, 1, 1, device=device)

D = Discriminator(nc, ndf).to(device)
G = Generator(nz, ngf, nc).to(device)

d_optimizer = torch.optim.RMSprop(D.parameters(), lr=lr)
g_optimizer = torch.optim.RMSprop(G.parameters(), lr=lr)

g_losses = []
d_losses = []
for epoch in range(epochs):
    for idx, (images, _) in enumerate(tqdm(dataloader, desc="Train", leave=False)):
        ## Update Critic : Maximize D(x) + D(G(z))
        d_optimizer.zero_grad()
        bs = images.size(0)

        x = images.to(device)
        Dx = D(x)

        z = torch.randn(bs, nz, 1, 1, device=device)
        Gz = G(z)

        d_real_loss = -torch.mean(Dx)
        d_fake_loss = torch.mean(D(Gz))
        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        d_optimizer.step()

        for p in D.parameters():
            p.data.clamp_(-clip_value, clip_value)

        if idx % critic_iter == 0:
            g_optimizer.zero_grad()

            Gz = G(z)
            g_loss = -torch.mean(D(Gz))
            g_loss.backward()
            g_optimizer.step()

            d_losses.append(d_loss.item())
            g_losses.append(g_loss.item())

    save_fake_images(epoch+1, G, fixed_noise)
    print(f"{epoch+1}/{epochs} : D Loss : {d_loss.item():.4f} G Loss : {g_loss.item():.4f}")

1/100 : D Loss : -1.2472 G Loss : 0.6623


2/100 : D Loss : -1.2042 G Loss : 0.6433


3/100 : D Loss : -0.5545 G Loss : 0.6649


4/100 : D Loss : -0.5780 G Loss : 0.6489


5/100 : D Loss : -0.7764 G Loss : 0.1936


...

96/100 : D Loss : -0.1941 G Loss : 0.2672


97/100 : D Loss : -0.1782 G Loss : 0.1609


98/100 : D Loss : -0.2094 G Loss : 0.2490


99/100 : D Loss : -0.1857 G Loss : -0.1843


100/100 : D Loss : -0.1988 G Loss : 0.3432
